# Analyse Biochemical data

In [1]:
import pandas as pd
import numpy as np 
import os 
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO
import warnings
from Bio import BiopythonParserWarning
warnings.simplefilter('ignore', BiopythonParserWarning)

In [2]:
def get_sequence(row, pdb = True)->str:
    """

        Get sequence from structure. Consider +-20 amino acids from 
        first and last isopeptide bond amino acid
    
    """
    structure_path = row["structure_path"]
    if pdb:
        r1 = row["Position 1\r\n(Bond 1)"]
        r2 = row["Position 2\r\n(catalytic)"]
        r3 = row["Position 3\r\n(Bond 2)"]
    else:
        r1 = row["r1_af"]
        r2 = row["r2_af"]
        r3 = row["r3_af"]
    seq_start = min([r1, r2, r3])
    seq_end = max([r1, r2, r3])
    sequence = list(SeqIO.parse(structure_path, "pdb-atom"))[0]
    # Adjust seq start and end based on pdb seq structure start and end
    pdb_start = sequence.annotations["start"]
    seq_start = seq_start - pdb_start - 20
    seq_end = seq_end - pdb_start + 20
    if seq_start < 0:
        seq_start = 0
    return str(sequence.seq)[seq_start:seq_end]

In [3]:
load_dotenv("../.env")
TABLE = os.getenv("TABLE")
PDB_BIOCHEM = os.getenv("PDB_BIOCHEM")
POSITIVE_CONTROL = os.getenv("POSITIVE_CONTROL")

In [4]:
df = pd.read_csv(TABLE)
df_bc = pd.read_csv(PDB_BIOCHEM)

df["structure_path"] = df.apply(lambda x:
                                os.path.join(POSITIVE_CONTROL, x["PDB code"]+"_"+x["Chain"]+".pdb"), axis=1)
df["sequence"] = df.apply(lambda x: get_sequence(x), axis=1)
df_bc = pd.merge(df_bc, df[["PDB code", "Chain", "Position 1\r\n(Bond 1)", "sequence"]])

## Cis/trans

In [19]:
cond1 = (df_bc["Is bonded"])
cond2 = (~df_bc["Interchain"])
cond3 = (~df_bc["Bad rotamer"])
cond4 = (df_bc["Isopeptide type"]!="Mutant")
mod_df = df_bc[cond1 & cond2 & cond3 & cond4].drop_duplicates("sequence").copy()
mod_df["Cis"] = False
mod_df.loc[(mod_df["pseudo_omega"]>-40)&(mod_df["pseudo_omega"]<40), "Cis"] = True
mod_df["Trans"] = False
mod_df.loc[(mod_df["pseudo_omega"]<-150)|(mod_df["pseudo_omega"]>150), "Trans"] = True
mod_df[mod_df["Isopeptide type"]=="CnaB-like"].value_counts(["Isopeptide type", "Cis", "Trans"], normalize=True).round(2)

Isopeptide type  Cis    Trans
CnaB-like        False  True     0.60
                 True   False    0.21
                 False  False    0.19
Name: proportion, dtype: float64